In [107]:
# detection 수행하는 부분
# 이 부분은 가볍게 detection 결과와 crop된 이미지가 잘 저장되는 지 확인하는 부분이다.
!python3 detect.py --weights /home/tlrkrwlsmd/yolov5/runs/train/exp2/weights/best.pt --source /home/tlrkrwlsmd/yolov5/samples/a2.PNG --save-crop --device 1

detect: weights=['/home/tlrkrwlsmd/yolov5/runs/train/exp2/weights/best.pt'], source=/home/tlrkrwlsmd/yolov5/samples/a2.PNG, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=1, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...
ERROR: Ignored the following versions that require a different python version: 10.0.0 Requires-Python >=3.8; 10.0.1 Requires-Python >=3.8; 10.1.0 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement Pillow>=10.0.1 (from versions: 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7.0, 1.7.1, 1.7.2, 1.7.3, 1.7.4, 1.7.5, 1.7.6, 1.7.7, 1.7.8, 2.0.0, 2.1.0

#### EasyOCR을 통한 텍스트 인식 부분
1. Detection 이후 인식이 되었다면 crop 디렉토리 생성.
2. 각 클래스별 crop 결과를 바탕으로 텍스트 인식 시작
3. 이름과 전화번호 중 하나라도 인식이 안 된다면 Detection failed 메세지를 뱉어낸다.

In [108]:
import os
import pandas as pd

# detect 하위 dir 경로 및 해당 dir name (concat)
subfolders = pd.DataFrame([f.path for f in os.scandir('/home/tlrkrwlsmd/yolov5/runs/detect/') if f.is_dir()], columns=['dir_path'])
subfolders_nm = pd.DataFrame([f.name for f in os.scandir('/home/tlrkrwlsmd/yolov5/runs/detect/') if f.is_dir()], columns=['dir_name'])
subfolders_frm = pd.concat([subfolders, subfolders_nm], axis=1)

# checkpoints dir 제거(사용할 최종 data) : sub_exp_frm
sub_exp_frm = subfolders_frm.loc[~subfolders_frm.dir_name.str.contains('check'), :]

# exp 뒤의 숫자가 가장 큰 dir 찾기 (가장 최신 detection) : new_dir
new_idx = sub_exp_frm.dir_name.str[3:].replace('', 0).astype(int).argmax()  # 해당 dir 있는 index number
new_dir = sub_exp_frm.iloc[new_idx, 0]

# 결과 저장 리스트
label_list = []
path_list = []

# 파일경로 저장할 딕셔너리
file_paths_dict = {'information': None, 'number': None, 'name': None}
if "crops" in os.listdir(new_dir):
    crops_dir = os.path.join(new_dir, "crops")
    for class_dir in os.listdir(crops_dir):
        class_path = os.path.join(crops_dir, class_dir)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith(".jpg"):
                    label_nm = class_dir  # 클래스 디렉터리 이름을 레이블로 사용
                    crop_img_path = os.path.join(class_path, filename)

                    label_list.append(label_nm)
                    path_list.append(crop_img_path)

                    # 각 클래스에 해당하는 딕셔너리의 키에 이미지 경로 추가
                    if label_nm == 'name':
                        file_paths_dict['name'] = crop_img_path
                    elif label_nm == 'number':
                        file_paths_dict['number'] = crop_img_path
                    elif label_nm == 'information':
                        file_paths_dict['information'] = crop_img_path

print(file_paths_dict)

# 'name', 'number', 'information' 중 모두가 검출되었는지 확인
if file_paths_dict['name'] and file_paths_dict['number'] and file_paths_dict['information']:
    print("detection success")
else:
    if not file_paths_dict['name']:
        print("Name detection failed")
    if not file_paths_dict['number']:
        print("Number detection failed")
    if not file_paths_dict['information']:
        print("Information detection failed")
    if not file_paths_dict['name'] and not file_paths_dict['number']:
        print("Name, Number detection failed")

{'information': '/home/tlrkrwlsmd/yolov5/runs/detect/exp11/crops/information/a2.jpg', 'number': '/home/tlrkrwlsmd/yolov5/runs/detect/exp11/crops/number/a2.jpg', 'name': '/home/tlrkrwlsmd/yolov5/runs/detect/exp11/crops/name/a2.jpg'}
detection success


In [109]:
import easyocr
from PIL import Image
from PIL.Image import Transpose
import numpy as np
import os

reader = easyocr.Reader(['ko', 'en'], gpu=True)

name_text = ""
number_text = ""
information_text = ""


for key, file_path in file_paths_dict.items():
    if file_path is not None and os.path.exists(file_path):
        image = Image.open(file_path)
        result = reader.readtext(image)
        print(result)
        # EasyOCR 상에서 맨 뒤의 값은 내부 유사도를 의미
        # 정확한 Recognition이 진행되면 유사도 값은 높게, 그렇지 않으면 낮은 값으로 나오게 된다.
        thres = 0.5
        if result[0][2] < thres:
            image = np.array(image.transpose(Transpose.ROTATE_180))
            result = reader.readtext(image)
        if key == 'name':
            name_text = result
        elif key == 'number':
            number_text = result
        elif key == 'information':  
            information_text = result
    else:
        print(f"Error: '{key}' 키에 대한 유효한 이미지 파일이 없습니다.")

# 'name'과 'number'가 없는 경우에 메시지 출력
if not name_text:
    print("Error: 'name' 키를 인식하지 못했습니다.")
if not number_text:
    print("Error: 'number' 키를 인식하지 못했습니다.")
else:
    print("Name:", name_text[0][1])
    print("Number:", number_text[0][1])
    if information_text:
        print("Information:", information_text[0][1])

[([[6, 10], [46, 10], [46, 34], [6, 34]], '사원', 0.9999983141264758)]
[([[11, 11], [131, 11], [131, 27], [11, 27]], '+82.10.8848.2705', 0.868036998836037)]
[([[10, 16], [104, 16], [104, 48], [10, 48]], '김 선준', 0.8028175234794617)]
Name: 김 선준
Number: +82.10.8848.2705
Information: 사원


In [110]:
import easyocr
from PIL import Image
import numpy as np
import os

reader = easyocr.Reader(['ko', 'en'], gpu=True)

name_text = ""
number_text = ""
information_text = ""

for key, file_path in file_paths_dict.items():
    if file_path is not None and os.path.exists(file_path):
        image = Image.open(file_path)
        
        # 이미지를 최대 3번까지 90도씩 돌려가며 OCR을 시도
        for _ in range(3):
            result = reader.readtext(np.array(image))  # NumPy 배열로 변환하여 easyocr에 전달
            print(result)
            thres = 0.7 if key == 'number' else 0.5  # 번호의 경우에만 thres 값을 높임
            if result[0][2] >= thres:
                break  # 인식 성공 시 루프 종료
            else:
                # 이미지를 90도 회전
                image = image.rotate(90, expand=True)

        if key == 'name':
            name_text = ''.join([item[1] for item in result])
            if len(name_text) >= 2 and len(name_text) <= 4:
                break
        elif key == 'number':
            number_text = result[0][1]
        elif key == 'information':  
            information_text = ''.join([item[1] for item in result])
            if len(information_text) == 1:  # 한 자리일 때
                for _ in range(3):
                    result = reader.readtext(np.array(image))
                    information_text += result[0][1]
                    if len(information_text) >= 2 and len(information_text) <= 4:
                        break
                    else:
                        image = image.rotate(90, expand=True)
    else:
        print(f"Error: '{key}' 키에 대한 유효한 이미지 파일이 없습니다.")

# 'name'과 'number'가 없는 경우에 메시지 출력
if not name_text:
    print("Error: 'name' 키를 인식하지 못했습니다.")
if not number_text:
    print("Error: 'number' 키를 인식하지 못했습니다.")
else:
    print("Name:", name_text)
    print("Number:", number_text)
    if information_text:
        print("Information:", information_text)

[([[6, 10], [46, 10], [46, 34], [6, 34]], '사원', 0.9999983141264758)]
[([[11, 11], [131, 11], [131, 27], [11, 27]], '+82.10.8848.2705', 0.8681945200896913)]
[([[10, 16], [104, 16], [104, 48], [10, 48]], '김 선준', 0.8018341064453125)]
Name: 김 선준
Number: +82.10.8848.2705
Information: 사원


In [113]:
# recognition_test
import os

# OCR 검증과정

# 이름 검증 - 각종 문자열 및 공백 제거
cleaned_name_text = ''.join(char for char in name_text if char.isalnum())

result_directory = '/home/tlrkrwlsmd/yolov5/result_txt'

# 각 디렉토리 경로 생성
name_directory = os.path.join(result_directory, 'name')
number_directory = os.path.join(result_directory, 'number')
information_directory = os.path.join(result_directory, 'information')

# 디렉토리 생성
os.makedirs(name_directory, exist_ok=True)
os.makedirs(number_directory, exist_ok=True)
os.makedirs(information_directory, exist_ok=True)

if 2 <= len(cleaned_name_text) <= 4:
    with open(os.path.join(name_directory, 'name_text_result.txt'), 'w', encoding='utf-8') as name_file:
        name_file.write(cleaned_name_text)
    print(cleaned_name_text)
    print("Name good")
else:
    print(cleaned_name_text)
    print("Name not good")
print()

# 번호 검증
cleaned_number_text = ''.join(char for char in number_text if char.isnumeric() or char == '+')

# 길이가 11자리이고 010으로 시작하거나, 길이가 13자리이고 +82로 시작하는 경우에 "OK" 출력
if (len(cleaned_number_text) == 11 and cleaned_number_text.startswith('010')) or (len(cleaned_number_text) == 13 and cleaned_number_text.startswith('+82')):
    with open(os.path.join(number_directory, 'number_text_result.txt'), 'w', encoding='utf-8') as number_file:
        number_file.write(cleaned_number_text)
    print(cleaned_number_text)
    print("Number good")
else:
    print(cleaned_number_text)
    print("Number not good")

print()

# 정보 검증 - 각종 문자열 및 공백 제거
cleaned_information_text = ''.join(char for char in information_text if char.isalnum())
if cleaned_information_text:  # 추가: 정보가 비어있지 않은 경우에만 파일 생성
    with open(os.path.join(information_directory, 'information_text_result.txt'), 'w', encoding='utf-8') as info_file:
        info_file.write(cleaned_information_text)
    print(cleaned_information_text)
    print("Information good")
else:
    print("Information not good")

김선준
Name good

+821088482705
Number good

사원
Information good


In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt

# # 이미지 파일 경로들을 리스트에 저장
# image_paths = ['/home/tlrkrwlsmd/yolov5/runs/detect/exp8/business_Card_test.png',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a2.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a3.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a6.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a7.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a8.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a9.PNG',
#                # '/home/tlrkrwlsmd/yolov5/runs/detect/exp8/a10.PNG']

# # 이미지를 subplot으로 표시할 행과 열의 수 설정
# rows = 2
# cols = 4

# # subplot 생성
# fig, axes = plt.subplots(rows, cols, figsize=(12, 6))

# # 이미지를 읽어서 subplot에 표시
# for i, ax in enumerate(axes.flat):
#     # 이미지 불러오기
#     img_path = image_paths[i]
#     img = Image.open(img_path)

#     # 이미지 표시
#     ax.imshow(img)
#     ax.axis('off')  # 좌표 축 숨김
#     ax.set_title(f'Image {i+1}')  # 각 subplot에 제목 추가

# # 레이아웃 조정
# plt.tight_layout()

# # 이미지 저장
# plt.savefig('subplot_images.png', bbox_inches='tight', pad_inches=0.1)

# # 그래프 표시
# plt.show()

In [105]:
pwd

'/home/tlrkrwlsmd/yolov5'